In [15]:
#check image shape
from PIL import Image

im = Image.open('C:/Users/clarq/Desktop/data/train/1aba91a601c6_01.jpg')

print(im.size)
print(type(im.size))

(1918, 1280)
<class 'tuple'>


In [16]:
#create convolutional net
import torch
import torch.nn as nn
import torchvision
import torch.nn.functional as F

In [17]:
class DoubleConv(nn.Module):
    def __init__(self, in_ch, out_ch):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, kernel_size=3, padding=1,stride=1), #two 3x3 conv
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),
            
            nn.Conv2d(out_ch, out_ch, 3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True))

    def forward(self, inputs):
        x = self.conv(inputs)
        
        #print(x.shape)
        return x
# if __name__=='__main__':
#     inputs=torch.rand((2,3,512,512))
#     c=DoubleConv(3,64) #build a class called DoubleConv, the class has input 3 and output 64
#     c(inputs)   # , and this is used on the inputs

In [18]:
#encoder
class Down(nn.Module):
    def __init__(self, in_ch, out_ch):
        super().__init__()
        self.conv = DoubleConv(in_ch, out_ch)
        self.pool = nn.MaxPool2d(kernel_size =2,stride=2)# kernel size is 2x2 for maxpool2d
        

    def forward(self, inputs):
        x = self.conv(inputs)
        p = self.pool(x)
       # print('conv',x.shape)
        #print('pool',p.shape)

        return x,p
# if __name__=='__main__':
#     inputs=torch.rand((2,3,512,512))
#     c=Down(3,64)
#     c(inputs)



In [19]:
#decoder
class Up(nn.Module):
    def __init__(self, in_ch, out_ch, bilinear=False):
        super(Up, self).__init__()

        if bilinear:
            self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        else:
            self.up = nn.ConvTranspose2d(in_ch , out_ch , kernel_size=2, stride=2,padding=0) 
        self.conv=DoubleConv(out_ch+out_ch , out_ch)# the input channel size of this conv net is the concatenation 
                                                    # channel size of the skip connection
                                                    # and the output of the decoder

    def forward(self, inputs, skip):#inputs and skip connections
        x_upsample = self.up(inputs)
        
        
        
        # the skip connection is larger than the spsampling output
        # so, (1)either the skip connection input needs to be cropped, or (2) the upsampled layer is padded to the size
        # of the skip connection (method (2)is used here)
        
        if x_upsample.shape != skip.shape:
            diffY = skip.size()[2]  - x_upsample.size()[2] # presumably, the skip connection is larger than the upsample
                                                       # diffY is the height difference between the two images, 
                                                       # skip.size()[2], [2] is the second element in skip.size()
                                                       # [a,b,c,d] a:batch size; b:channel; c:height; d:width
       
        
        
        
            diffX = skip.size()[3] - x_upsample.size()[3] 
            # presumably, the skip connection is larger than the upsample
            # diffX is the width difference between the two images
        
            x_padded = F.pad(x_upsample, (diffX // 2, diffX - diffX // 2,diffY // 2, diffY - diffY // 2),value=0)
            # zero padding,'//' means divide and take the whole value of the output only refer to 
            # onenote 'PhD lit - explain F.pad'
        else:
            x_padded=x_upsample
        
        
        x_concatenate = torch.cat([x_padded,skip], dim=1)# concatenates the channels (dim=1, i.e., the second 
                                       # element in the shape bracket) of the skip and the output of the decoder(up)
        x = self.conv(x_concatenate)
        
        return x



# if __name__=='__main__':
#     inputs=torch.rand((2,64,256,256))
#     skip=torch.rand((2,32,512,512))# this should the size of the image in the previous layer
#     c=Up(64,32)
#     print(c(inputs,skip).shape)
    


In [20]:
# output of the unet with convolutional net (traditional CNN) no relu in the final 1X1 covolutional
class OutConv(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(OutConv, self).__init__()
        self.conv = nn.Conv2d(in_ch, out_ch, kernel_size= 1, padding=0)# when kernel_size=1, padding always 0

    def forward(self, x):
        x = self.conv(x)
        return x

In [23]:
#start to build the whole Unet architecture
#change the input feature at self.down1 
#change the output feature(classes) at self.outc

class Unet(nn.Module):
    def __init__(self):
        super().__init__()
#         self.n_channels = in_channels
#         self.n_classes =  classes

        '''Input'''
       # self.inc = InConv(1, 64)
        
        '''Encoder'''
        self.down1 = Down(3, 64)#encoder layer1,1 is the input feature channel,  you can change         
        self.down2 = Down(64, 128)#encoder layer2
        self.down3 = Down(128, 256)#encoder layer3
        self.down4 = Down(256, 512)#encoder layer4
        
        '''Bottleneck'''
        self.bottleneck = DoubleConv(512, 1024) #bottleneck, no skip connection, the bottom of the 'U-shape'
        
        '''Decoder'''
        self.up1 = Up(1024, 512)#decoder layer1
        self.up2 = Up(512,256)#decoder layer2
        self.up3 = Up(256, 128)#decoder layer3
        self.up4 = Up(128, 64)#decoder layer4
        
        
        '''classifier'''
        self.outc = OutConv(64, 1)# the '3' is the number of classes. if binary classification, it is 1

    def forward(self, inputs):
        s1,p1 = self.down1(inputs)# s is the output dimensions after the skip connection concatenated
                                  # p is the output dimension from the maxpooling layer
        s2,p2 = self.down2(p1)
        s3,p3 = self.down3(p2)
        s4,p4 = self.down4(p3)
        print('s1',s1.shape,'p1',p1.shape)
        print('s2',s2.shape,'p2',p2.shape)
        print('s3',s3.shape,'p3',p3.shape)
        print('s4',s4.shape,'p4',p4.shape)
        b = self.bottleneck(p4)
        print(b.shape)
        d1 = self.up1(b, s4)
        d2 = self.up2(d1, s3)
        d3 = self.up3(d2, s2)
        d4 = self.up4(d3, s1)
        output = self.outc(d4)
        #print('d4',d4.shape)
        return output


if __name__=='__main__':
    inputs=torch.rand((2,3,520,520))
    model=Unet()
    y=model(inputs)
    print('y',y.shape)

s1 torch.Size([2, 64, 520, 520]) p1 torch.Size([2, 64, 260, 260])
s2 torch.Size([2, 128, 260, 260]) p2 torch.Size([2, 128, 130, 130])
s3 torch.Size([2, 256, 130, 130]) p3 torch.Size([2, 256, 65, 65])
s4 torch.Size([2, 512, 65, 65]) p4 torch.Size([2, 512, 32, 32])
torch.Size([2, 1024, 32, 32])
y torch.Size([2, 1, 520, 520])


In [8]:
# def test():
#     x=torch.rand((3,1,160,160))
#     model=Unet()
#     y=model(x)
#     print('y',y.shape)
#     print('x',x.shape)
#     assert y.shape == x.shape

# if __name__=='__main__':    
#     test()

In [9]:
# data loader 
import os
from PIL import Image 
# pillow library
import numpy as np
from torch.utils.data import Dataset

#dataset class,this is classic
class T_Dataset(Dataset):
    def __init__(self,image_directory,mask_directory,transform=True):
        # image_directory is the image path,transform is augmentations,
        # mask_directory can be changed to targets if it is image classification
        self.image_directory = image_directory
        self.mask_directory = mask_directory
        self.transform = transform
        self.images = os.listdir(image_directory)
        #os.listdir() method in python is used to get the list of all files and directories
        #in the specified directory. If we don’t specify any directory, 
        #then list of files and directories in the current working directory will be returned.
    
    
    def __len__(self):
        return len(self.images)# size of image dataset 
    
    def __getitem__(self,index): # index iterates from 0 to number of dataset
        img_path=os.path.join(self.image_directory, self.images[index])
        #os.path.join() method in Python join one or more path components intelligently
        mask_path=os.path.join(self.mask_directory, self.images[index].replace('.jpg','_mask.gif'))
                                                                             # in this mask，
                                                                             # white is labeled as 255.0 
                                                                             #and black is labeled as 0.0 
                
        mask[mask == 255.0] =1.0 # preprocess for the mask, since sigmoid activation is used, so transfer 255 to 1
        
        
        #image=np.transpose(image,(2,0,1).astype(np.float32)) # change the array of image to channel first
        #tensor.unsqueeze(0) #if the iamge is greyscale, you need to add one dimension to it
        if self.transform is not None:# data augmentaion
            augmentations =self.transform(image = image, mask = mask)
            image = augmentations['image']
            mask = augmentations ['mask']
        return image,mask
            
            

In [10]:
# create training part
import albumentations as A
from albumentations.pytorch import ToTensorV2
from tqdm import tqdm #loading progress bar
import torch.optim as optim

#save & load mode
def save_checkpoint(state,filename='my_checkpoint.pth.tar'):
    print('=> saving checkpoint')
    torch.save(state,filename)

def load_checkpoint(checkpoint,model):
    print('=> loading checkpoint')
    model.load.state.dict(checkpoint['state_dict'])

def check_accuracy(loader,model,device='cuda'):
    num_correct=0
    num_pixels=0
    dice_score=0
    model.eval()
    
    with torch.no_grad():
        for image,mask in loader:
            images=image.to(device)
            masks=mask.to(device).unsqueeze(1) # since this is greyscale so add one dimension with unsqueeze
            preds=torch.sigmoid(model(images))
            preds=(preds>0.5).float()
            num_correct+=(preds==masks).sum()
            num_pixels+=torch.numel(preds)#numel: number of elements
            dice_score+= (2*(preds*masks).sum())/((preds+masks).sum()+1e-8) 
            # this is for binary to evaluate the output, google for multiclass
    print(f' got{num_correct}/{num_pixels} with acc {num_correct/num_pixels*100:.2f}' ) 
    print(f' Dice score:{dice_score/len(loader)}' )
    model.train()

def save_predictions_as_imgs(loader,model,folder='C:/Users/clarq/Desktop/data/saved images',device='cuda'):
    model.eval()
    for idx,(images,masks) in enumerate(loader):
        images=images.to(device=Device)
        with torch.no_grad():
            preds=torch.sigmoid(model(images))
            preds=(preds>0.5).float()
        torchvision.utils.save_image(
            preds,f'{folder}/pred_{idx}.png')
        torchvision.utils.save_image(masks.unsqueeze(1),f'{folder}/true_{idx}.png')
        
    model.train() 

In [11]:
# hyperparameters
learning_rate=1e-4
Device='cuda' if torch.cuda.is_available() else 'cpu'
Batch_size=16
num_epochs=3
num_workers=0
Image_height=160  # 1280 originally, use 160 in this example(only use a small part of the image),
                # in real competition, use 1280
Image_width=240 #1918 originally
Pin_memory= True
Load_model= False
val_percent: float = 0.3
train_image_directory='C:/Users/clarq/Desktop/data/train'
train_mask_directory='C:/Users/clarq/Desktop/data/train_masks'
# val_img_directory='C:/Users/clarq/Desktop/data/val'
# val_mask_directory='C:/Users/clarq/Desktop/data/val_masks'

In [12]:
#data augmentation,create model
from torch.utils.data import DataLoader
def main():
    train_transform = A.Compose([
        A.Resize(height=Image_height, width= Image_width),
        A.Rotate(limit=35,p=1.0),
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.1),
        A.Normalize(mean=[0,0,0],std=[1,1,1],max_pixel_value=225),
        ToTensorV2(),
    ])
#     val_transform = A.Compose([
#         A.Resize(height=Image_height, width= Image_width),
#         A.Normalize(mean=[0,0,0],std=[1,1,1],max_pixel_value=225),
#         ToTensorV2(),
#     ])
    


    model=Unet().to(Device)
    loss_function=nn.BCEWithLogitsLoss()# bianry cross entropy, sigmoid is included in the loss function
                                         # use CROSS entropy loss for multiple classification

    optimizer= optim.Adam(model.parameters(),lr=learning_rate)
    

    ds=T_Dataset(image_directory=train_image_directory,mask_directory=train_mask_directory,
                     transform=train_transform)

    
    n_val = int(len(ds) * val_percent)
    n_train = len(ds) - n_val
    train_set, val_set = torch.utils.data.random_split(ds, [n_train, n_val],
                                                   generator=torch.Generator().manual_seed(0))
    train_loader = DataLoader(train_set, shuffle=True,batch_size=Batch_size,pin_memory=Pin_memory,num_workers=0)
    val_loader = DataLoader(val_set, shuffle=False, batch_size=Batch_size,pin_memory=Pin_memory,num_workers=0)
      
    
    if Load_model: #Load model = False as set, change to True to use this if loop
        load_checkpoint(torch.load('my_checkpoint.pth.tar'),model)
        
    check_accuracy(val_loader,model,device=Device)
        
        
        
    scaler=torch.cuda.amp.GradScaler()
    
  
    for epoch in range(num_epochs):
        loop=tqdm(train_loader)    
        for batch_idx, (data,targets) in enumerate(loop):
            data= data.to(device=Device)# data is the input image
            targets=targets.float().unsqueeze(1).to(device=Device)#target is the masks
            #forward
            with torch.cuda.amp.autocast():
                predictions=model(data)
                loss=loss_function(predictions,targets)
            
        
        
            #backward
            optimizer.zero_grad()
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
        
            #update tadm loop
            loop.set_postfix(loss=loss.item())
         
        #save model
        checkpoint={'state_dict': model.state_dict(),'optimizer':optimizer.state_dict(),}
        save_checkpoint(checkpoint)
        
        #check accuracy
        check_accuracy(val_loader, model,device=Device)
        
        # print some exmaples to the folder
        save_predictions_as_imgs(val_loader,model,folder='C:/Users/clarq/Desktop/data/saved images',device=Device)
  
    
   
    print('Finished Training')  
    
        
if __name__=='__main__':
    main()

 got42117941/53376000 with acc 78.91
 Dice score:0.0


100%|████████████████████████████████████████████████████████████████████| 203/203 [02:46<00:00,  1.22it/s, loss=0.158]


=> saving checkpoint
 got52639141/53376000 with acc 98.62
 Dice score:0.9666340351104736


100%|████████████████████████████████████████████████████████████████████| 203/203 [02:34<00:00,  1.31it/s, loss=0.115]


=> saving checkpoint
 got52863339/53376000 with acc 99.04
 Dice score:0.9771881103515625


100%|███████████████████████████████████████████████████████████████████| 203/203 [02:30<00:00,  1.35it/s, loss=0.0789]


=> saving checkpoint
 got52955507/53376000 with acc 99.21
 Dice score:0.9813230037689209
Finished Training


In [13]:
# import matplotlib.pyplot as plt
# ds=T_Dataset(image_directory=train_image_directory,mask_directory=train_mask_directory,
#                      transform=train_transform)
# data = torch.utils.data.DataLoader(ds, batch_size=Batch_size,
#                                    pin_memory=Pin_memory, shuffle=True,num_workers=0)


# def imshow(img):
# #    img = img / 2 + 0.5  # unnormalize
#     npimg = img.numpy()
#     plt.imshow(np.transpose(npimg, (1, 2, 0)))
#     plt.show()

# # get some random training images
# dataiter = iter(data)
# images, labels = dataiter.next()

# # show images
# imshow(torchvision.utils.make_grid(images))
# print(images.shape)

In [14]:
# '''Dataloader'''
# class SynDataset(Dataset):
#     def __init__(self,path,transforms=None):
#         self.image_list = listdir(path)
#         #listdir() method in python is used to get the list of all files and directories
#         #in the specified directory. If we don’t specify any directory, 
#         #then list of files and directories in the current working directory will be returned.
#         self.path = path
#         self.transforms=transforms
    
#     def __len__(self):
#         return len(self.image_list)# size of image dataset 
  
#     def __getitem__(self,idx): # index iterates from 0 to number of dataset
#         img = np.load(self.path+self.image_list[idx])
        
#         im, la = img['image'], img['label']
       
#         sample = {'image': im, 'label': la} #make a dictionary
    
#         if self.transforms is not None:
#             sample=self.transforms(sample) #在后面调用SynDataset train model 的时候会define 具体的 transform 
        
#         return sample

    
    
# '''data augmentation'''
# class Resize(object):
#     def __init__(self, output_size):
        
#         self.size = output_size
      
#     def __call__(self, sample):
#         image, label = sample['image'], sample['label'] #call the dictionary sample['image']= im
    
#         im = transform.resize(image, (self.size, self.size))
#         la = transform.resize(label, (self.size, self.size),order=0,anti_aliasing=False)
        
#         return {'image': im, 'label': la}
    
# class Rotate(object):

#     def __init__(self, angle):
        
#         self.angle = angle
      
#     def __call__(self, sample):
#         image, label = sample['image'], sample['label']
        
#         if np.random.rand()<0.25:        
#             alpha = np.random.randint(-self.angle, self.angle)
#             image = transform.rotate(image, alpha, resize=False)
#             label = transform.rotate(label, alpha, resize=False)
        
#         return {'image': image, 'label': label}
    
# class flip(object):
    
#     def __call__(self, sample):
#         image, label = sample['image'], sample['label']
            
#         if np.random.rand()<0.25: 
#             image = np.flipud(image).copy()
#             label = np.flipud(label).copy()
#         elif np.random.rand()<0.25: 
#             image = np.fliplr(image).copy()
#             label = np.fliplr(label).copy()
            
#         return {'image': image, 'label': label}


# """
# load the train data and train the network
# """

# train_path = '/home/dewolf151/train_npz/'
# train_files = listdir(train_path)
# batch_size = 24
# num_epochs = 250 

# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# print('the available device is:', device, flush=True)

           
# composed=transforms.Compose([Resize(224),
#                           Rotate(15),
#                           flip()])

# trainloader=DataLoader(SynDataset(train_path,composed),batch_size=batch_size,shuffle=True)

# Full_Rnet = torch.hub.load('pytorch/vision:v0.6.0', 'resnet50', pretrained=True)
# model = TransUnet(Full_Rnet)
# model.to(device)

# ce_loss = torch.nn.CrossEntropyLoss()
# optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=0.0001)

# store_loss = np.zeros((num_epochs))
# acc = []

# for epoch in range(num_epochs):
#   epoch_loss = 0
#   for ii, data in enumerate(trainloader):
#     image, labels = data['image'].to(device), data['label'].to(device)
    
#     im_in = image.unsqueeze(1)
#     prediction = model(im_in)
    
#     loss = ce_loss(prediction, labels[:].long())
    
#     optimizer.zero_grad()
#     loss.backward()
#     optimizer.step()
    
#     epoch_loss += loss.item()    
        
#   avg_loss = epoch_loss / (ii+1)
#   print('epoch %d loss: %.3f' % (epoch + 1, avg_loss ), flush=True)
    
#   store_loss[epoch] = avg_loss
  
#   if epoch % 5 == 0:
#       max_pred = torch.argmax(prediction,1)
#       num_pix = labels.nelement()
#       corr = max_pred.eq(labels).sum().item()
#       acc.append(100 * corr / num_pix)
      
        
# torch.save(model.state_dict(), 'saved_models/mod.pth')

# np.save('saved_data/train_acc',acc)
# np.save('saved_data/train_loss',store_loss)

# """
# Define the metrics used to quantify the performance during the testing time
# """

# def diceCoef(pred, gt):
#     nclas = 8 #the number of classes except background
#     N = gt.size(0) #the batch size
    
#     pred_flat = pred.view(N, -1)
#     gt_flat = gt.view(N, -1)
    
#     Dice = np.zeros((N,nclas))
#     for ii in range(nclas):
#         logit_pred = pred_flat == ii+1
#         logit_gt = gt_flat == ii+1
     
#         intersection = (logit_pred * logit_gt).sum(1)
#         unionset = logit_pred.sum(1) + logit_gt.sum(1)
#         Dice[:,ii] = 2 * (intersection) / (unionset)
    
#     #Dice[np.isnan(Dice)] = 0 #remove the Nan when class is not present
#     return 100*Dice

# def Hausdorff(pred, gt):
#     batch_s = pred.size(0)
#     HDC = np.zeros(batch_s)
    
#     pred[pred>0] = 1
#     pred = pred.detach().cpu().numpy()
    
#     gt[gt>0] = 1
#     gt = gt.detach().cpu().numpy()
    
#     for jj in range(batch_s): 
#         if pred[jj,:,:].sum() > 0 and gt[jj,:,:].sum()>0:
#             HDC[jj] = binary.hd95(pred[jj,:,:], gt[jj,:,:])
#         else:
#             HDC[jj] = 0
        
#     return HDC

# """
# Rescale the input images back for the test data set
# """
# def Test_rescale(sample,size):

#     image, label = sample['image'][:], sample['label'][:]
    
#     N = image.shape[0]
#     im = transform.resize(image, (N, size, size))
#     la = transform.resize(label, (N, size, size),order=0,preserve_range=True,anti_aliasing=False).astype(np.uint8)
        
#     return im, la

# """
# Load the test data and run it through the model
# """

# #Full_Rnet = torch.hub.load('pytorch/vision:v0.6.0', 'resnet50', pretrained=True)
# #model = TransUnet(Full_Rnet)
# #model.to(device)
# #model.load_state_dict(torch.load('saved_models\\mod.pth'))

# test_path = '/home/dewolf151/test_vol_h5/'
# test_files = listdir(test_path)  

# HD = []
# first_it = True
# for nn in range(len(test_files)):

#     test_data = h5py.File(test_path + test_files[nn])
    
#     T_image, T_label = Test_rescale(test_data,224)
    
#     #only send the images to gpu for running it through network
#     dataset = torch.utils.data.TensorDataset(torch.Tensor(T_image).to(device), 
#                                              torch.Tensor(T_label) )
        
#     testloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
#                                               shuffle=False, num_workers=0)
    
    
#     for i, data in enumerate(testloader):
#         image, labels = data
        
#         im_in = image.unsqueeze(1)
#         prediction = model(im_in)
        
#         prediction = torch.argmax(prediction, dim=1).cpu() #get tensor back to cpu
        
                    
#         if first_it:
#             DSC = diceCoef(prediction, labels)
#             first_it = False
#         else:
#             DSC = np.concatenate((DSC, diceCoef(prediction, labels)), axis=0)
            
#         HD = np.append(HD, Hausdorff(prediction, labels))
        
# np.save('saved_data/Dice_scores',DSC)
# np.save('saved_data/HD_scores',HD)

# print('training and testing has finished', flush=True)
        